<a href="https://colab.research.google.com/github/dubisx/EMCT_final/blob/main/2.%20tf_transformer_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 7.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import MIDITokenizer, REMI
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [ ]:
attn_window = 25
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
directory = "/content/gdrive/MyDrive/ala/REMI/tokens_BPE/"

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file                             

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

Streaming output truncated to the last 5000 lines.
batch number: 435890
input: [35, 913, 63, 432, 350, 32, 575, 343, 39, 698, 51, 94, 131, 61, 346, 342, 44, 432, 887, 679, 44, 94, 131, 64, 346]
target: [913, 63, 432, 350, 32, 575, 343, 39, 698, 51, 94, 131, 61, 346, 342, 44, 432, 887, 679, 44, 94, 131, 64, 346, 347]
-----
batch number: 435891
input: [94, 131, 61, 346, 342, 44, 432, 887, 679, 44, 94, 131, 64, 346, 347, 35, 913, 350, 32, 575, 44, 94, 127, 343, 39]
target: [131, 61, 346, 342, 44, 432, 887, 679, 44, 94, 131, 64, 346, 347, 35, 913, 350, 32, 575, 44, 94, 127, 343, 39, 698]
-----
batch number: 435892
input: [131, 64, 346, 347, 35, 913, 350, 32, 575, 44, 94, 127, 343, 39, 698, 46, 94, 127, 64, 346, 342, 47, 94, 131, 63]
target: [64, 346, 347, 35, 913, 350, 32, 575, 44, 94, 127, 343, 39, 698, 46, 94, 127, 64, 346, 342, 47, 94, 131, 63, 346]
-----
batch number: 435893
input: [127, 343, 39, 698, 46, 94, 127, 64, 346, 342, 47, 94, 131, 63, 346, 887, 679, 347, 35, 913, 63, 432, 350

In [ ]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 437140
target_seq length: 437140


In [ ]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [ ]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 349712
train target: 349712
val input: 43714
val target: 43714
test input: 43714
test target: 43714


(None, None)

In [ ]:
#creates tokenizer
tokenizer = REMI(params = "/content/gdrive/MyDrive/ala/REMI/params.json")

In [ ]:
# Creates model
config = GPT2Config( 
    vocab_size=1000,
    n_positions=attn_window,
    n_embd=64,
    n_layer=4,
    n_head=4,
    n_inner=128,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    pad_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)                      

In [ ]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [ ]:
num_epoch = 150

# Create a callback that saves the model's weights
cp_callback = [
tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_25attn')),
tf.keras.callbacks.LambdaCallback(on_epoch_end=lambda epoch, logs: model.save_weights('/content/gdrive/MyDrive/model_f/REMI_weights_25attn'))

]


history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))


Epoch 1/150
10929/10929 [==============================] - 294s 25ms/step - loss: 5.0721 - accuracy: 0.0701 - val_loss: 4.0279 - val_accuracy: 0.1373
Epoch 2/150
10929/10929 [==============================] - 258s 24ms/step - loss: 3.7634 - accuracy: 0.1767 - val_loss: 3.3999 - val_accuracy: 0.2292
Epoch 3/150
10929/10929 [==============================] - 251s 23ms/step - loss: 3.3559 - accuracy: 0.2431 - val_loss: 3.0917 - val_accuracy: 0.2918
Epoch 4/150
10929/10929 [==============================] - 253s 23ms/step - loss: 3.1317 - accuracy: 0.2869 - val_loss: 2.9209 - val_accuracy: 0.3221
Epoch 5/150
10929/10929 [==============================] - 252s 23ms/step - loss: 3.0039 - accuracy: 0.3107 - val_loss: 2.8102 - val_accuracy: 0.3430
Epoch 6/150
10929/10929 [==============================] - 283s 26ms/step - loss: 2.9201 - accuracy: 0.3260 - val_loss: 2.7424 - val_accuracy: 0.3536
Epoch 7/150
10929/10929 [==============================] - 326s 30ms/step - loss: 2.8629 - accuracy:

In [ ]:
model.save_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_25attn', saved_model = True)
model.save_weights('/content/gdrive/MyDrive/model_f/REMI_weights_25attn')

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model1 = TFGPT2LMHeadModel(config)     

model1.from_pretrained('/content/gdrive/MyDrive/model_f/REMI_FD_model_25attn')
model1.load_weights('/content/gdrive/MyDrive/model_f/REMI_weights_25attn')    

model1.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
model1.evaluate(x_test, y_test)